## 라이브러리 불러오기

In [ ]:
import random
import pandas as pd
import numpy as np
import os

# 사이킷런에서 랜덤포레스트 모델 불러오기
from sklearn.ensemble import RandomForestRegressor

# 경고 메세지 무시하기
import warnings
warnings.filterwarnings(action='ignore') 

In [ ]:
import matplotlib.pyplot as plt

# 브라우저에서 바로 그려지도록(그래프가 화면에 뜨지 않을 때)
%matplotlib inline

# 그래프(figure)에 retina display 적용
%config InlineBackend.figure_format = 'retina'

# Colab 의 한글 폰트 설정(나눔고딕체)
plt.rc('font', family='NanumGothic')

# 조용하 강사님이 올려주신 코드(한글화-matplotlib)
!pip install koreanize_matplotlib
import koreanize_matplotlib

## 데이터 불러오기

In [ ]:
train = pd.read_csv('train.csv',encoding='cp949')
test = pd.read_csv('test.csv',encoding='cp949')
submission=pd.read_csv('sample_submission.csv',encoding='cp949')

In [ ]:
# 컬럼명 변환을 위한 리스트 할당
train_col = ['num', 'date_time', 'power', 'temp', 'wind', 'hum', 'rain', 'sun', 'cooler', 'solar']
test_col = ['num', 'date_time', 'temp', 'wind', 'hum', 'rain', 'sun', 'cooler', 'solar']
train.columns=train_col
test.columns=test_col

## 전처리

### 날짜 전처리

- 파생변수, 데이터 타입 변경

In [ ]:
def make_time(train):
    """
    시간 관련 변수를 추가하기 위한 함수
    """
    train['date_time'] = pd.to_datetime(train.date_time)
    
    train['month'] = train.date_time.dt.month                    # 월(숫자)
    train['day'] = train.date_time.dt.day                        # 일(숫자)
    train['hour'] = train.date_time.dt.hour                      # 시(숫자)
    train['weekday'] = train.date_time.dt.weekday                # 요일(숫자)
    train['dayofyear'] = train.date_time.dt.dayofyear            # 연 기준 몇일째(숫자)
    #train['week']=train.date_time.dt.weekofyear
    
    ## cyclical encoding
    train['sin_time'] = np.sin(2*np.pi*train.hour/24)
    train['cos_time'] = np.cos(2*np.pi*train.hour/24)
    
    ### 공휴일 변수 추가
    train['holiday'] = train.apply(lambda x : 0 if x['day']<5 else 1, axis = 1)
    train.loc[('2020-08-17'<=train.date_time)&(train.date_time<'2020-08-18'), 'holiday'] = 1
    
    
    ### 날씨 변수 추가
    train['THI'] = 1.9*train['temp'] - 0.55*(1-train['hum'])*(1.8*train['temp']-26)+32                                # 불쾌지수
    train['chill']=13.12+0.6215*train['temp']-11.37*train['wind']**0.16+0.3965*train['wind']**0.16*train['temp']        # 체감온도
    
    train['냉방도시'] = 0  # 냉방이 필요하지 않은 도시로 초기화

    # 여름 동안 냉방이 필요한 도시로 설정
    train.loc[(train['temp'] > 30) & (train['hum'] > 70), '냉방도시'] = 1
    
    return train

### 날씨 전처리

- 파생변수, 결측치

In [ ]:
def make_nan(test):
    """
    날씨 관련 결측치를 처리하기 위한 함수
    """
    
    # 결측치 처리
    test['wind']=test['wind'].interpolate(method='linear')
    test['rain']=test['rain'].interpolate(method='linear')
    test['sun']=test['sun'].interpolate(method='linear')
    
    test['temp']=test['temp'].interpolate(method='polynomial',order=3).fillna(method='ffill')
    test['hum']=test['hum'].interpolate(method='polynomial',order=3).fillna(method='ffill')
    
    test['cooler']=test['cooler'].fillna(0)
    test['solar']=test['solar'].fillna(0)
    
    return test

In [ ]:
def make_weather(data):
    weather=data.groupby(['num','dayofyear'])['temp'].agg({'max','min','mean'}).reset_index()
    weather['temp_range']=weather['max']-weather['min']
    
    data=pd.merge(data,weather,on=['num','dayofyear'],how='left')
    return data
    

In [ ]:
train = make_time(train)
test=make_nan(test)
test = make_time(test)
train = make_weather(train)
test=make_weather(test)

### 전력사용량 전처리

- 파생변수

In [ ]:
from tqdm import tqdm

def make_power(data):
    """
    전력 사용량 관련 변수를 추가하기 위한 함수
    """
    
    # 건물별, 요일별, 시간별 발전량 평균
    power_mean = pd.pivot_table(data, values = 'power', index = ['num', 'hour', 'day'], aggfunc = np.mean).reset_index()
    tqdm.pandas()
    data['day_hour_mean'] = data.progress_apply(lambda x : power_mean.loc[(power_mean.num == x['num']) & (power_mean.hour == x['hour']) & (power_mean.day == x['day']) ,'power'].values[0], axis = 1)
    
    # 건물별 시간별 발전량 평균 넣어주기
    power_hour_mean = pd.pivot_table(data, values = 'power', index = ['num', 'hour'], aggfunc = np.mean).reset_index()
    tqdm.pandas()
    data['hour_mean'] = data.progress_apply(lambda x : power_hour_mean.loc[(power_hour_mean.num == x['num']) & (power_hour_mean.hour == x['hour']) ,'power'].values[0], axis = 1)
    
    # 건물별 시간별 발전량 표준편차 넣어주기
    power_hour_std = pd.pivot_table(data, values = 'power', index = ['num', 'hour'], aggfunc = np.std).reset_index()
    data['hour_std'] = data.progress_apply(lambda x : power_hour_std.loc[(power_hour_std.num == x['num']) & (power_hour_std.hour == x['hour']) ,'power'].values[0], axis = 1)
    return data

In [ ]:
train = make_power(train)

# 횡단면 분석

## Ridge 모델

## Lasso 모델

## Elastic 모델

## Xgboost 모델

## Gradient Boosting

## Catboost모델

# 시계열 관련 모델

## Arima

# Prophet모델